In [ ]:
import Logging: global_logger
import TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

In [ ]:
using ModelingToolkit 
using DomainSets
using MethodOfLines
using NonlinearSolve
using DifferentialEquations
include("multiharmonic_balance.jl");
include("jacobian.jl");
using Random
using Base.Threads
using LinearSolve
using IncompleteLU

# Plotting
using Plots

# FFTW 
using FFTW

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
t0::Float64 = 0.0;
T::Float64 = 200.0;
Nx::Int64 = N = 200;
stepx = (xright - xleft)/Nx;
L = xright - xleft;
dt = 1e-3
Nt::Int64 = (T - t0)/dt;
grid = collect(xleft:stepx:xright);

#Definition of constants specific to the equation (Wave Equation) (placeholders):
gamma = 0.0;
gamma3_final = 1.0;

g0::Float64 = 1; # m / s^2
height = 1; # m
harmonics = 2; # number of harmonics
A_forcing::Float64 = 200
lambda_forcing::Float64 = -40;
omega = 2.0;

#Define parameters and variables
@parameters x y t
@variables u_FD(..)

#Define differential operators:
const Dt = Differential(t);
const Dx = Differential(x);
const Dy = Model.Differential(y)
const Dtt = Differential(t)^2;
const Dxx = Differential(x)^2;

In [ ]:
#Define initial condition (placeholder sinusoidal):
u0_FD(x, t) = 0
v0_FD(x) = 0


#Define equation (with and without cubic damping term):
equation = [Dtt(u_FD(x,t)) + gamma*Dt(u_FD(x,t)) + gamma3_final*(Dt(u_FD(x,t)))^3 ~ g0*height*Dxx(u_FD(x,t)) - A_forcing * exp(-40*x^2) * sin(omega*t)]
#equation = [Dtt(u(x,t)) ~ c^2*Dxx(u(x,t))]


#Define domains:
domains = [x ∈ Interval(xleft, xright), t ∈ Interval(t0, T)]

#Define boundary conditions:
bound_conditions = [u_FD(x, 0) ~ u0_FD(x, 0),
                     u_FD(xleft, t) ~ 0,
                     (u_FD(xright, t)) ~ 0,
                     Dt(u_FD(x, 0)) ~ v0_FD(x)]

#Create PDE system and assign identifier (order matters: eq, bcs, domains, paramers, variables; format parameters and variables as lists, provide arguments whenever they exist):
@named pde_sys = PDESystem(equation, bound_conditions, domains, [x, t], [u_FD(x,t)])

#Define a discretization (first argument specifies parameters to discretize, second indicates parameters to keep continuous, third provides the approximation order):
discretization = MOLFiniteDifference([x=>grid], t, approx_order=2)

#Proceed with discretization:
@time eq_discretized = discretize(pde_sys,discretization)

#Solve the resulting system (specified solver dictates whether or not time marching is used, as well as the method: Euler(), ImplicitEuler(), or Trapezoidal(), all with given time steps):
#Warning: if the time step is not explicitly given, solver may default to actual method of lines, instead of time marching\

solutionFD = solve(eq_discretized, ImplicitEuler(), dt=dt, saveat=0.01, progress=true, progress_steps=200);

# Your solution data
discrete_t = solutionFD.t
sol_u = solutionFD[u_FD(x, t)]

print(solutionFD.retcode);

## Visualization FD

In [ ]:
# Create the animation object
anim = @animate for k in 18000:20000
    t = discrete_t[k]
    plot(
        grid, sol_u[:, k],
        ylim = (-.01, .01),
        xlabel = "x",
        ylabel = "u(x, t)",
        title = "t = $(round(t, digits=2))",
        legend = false
    )
end

# Save the animation as a GIF
gif(anim, "solution_animation_1.gif", fps = 60)

# Harmonic Balance

In [ ]:
function make_WaveEquation_1D(u_HB, g0, height, gamma, gamma3, forcing)
            
    pde::Symbolics.Num = Dt(Dt(u_HB)) - g0*height*(Dx(Dx(u_HB))) + gamma*Dt(u_HB) + gamma3*Dt(u_HB)*Dt(u_HB)*Dt(u_HB) - forcing;

    return pde
end;

In [ ]:
u0 = rand((Nx + 1) * harmonics * 2)*7;
solHB = nothing 
u_hist = Vector{Float64}[]  # store history for prediction
gamma3_hist = Float64[]


vars, var_exprs, (u_HB,) = create_ansatz((x,), t, omega, harmonics);
    
forcing = A_forcing * exp(lambda_forcing*x^2) * sin(omega * t)

bc_conds = Dict(
    :A1 => [:(0.0), 0.0],
    :B1 => [:(0.0), 0.0],
    :A2 => [:(0.0), 0.0],
    :B2 => [:(0.0), 0.0],
    :A3 => [:(0.0), 0.0],
    :B3 => [:(0.0), 0.0],
)

gamma3 = 0.0
dgamma3 = 0.1
resid = nothing

while gamma3 < gamma3_final
    timed_out = false
    println("Beginning the symbolics manipulation")
    flush(stdout)
    
    if length(u_hist) >= 2
        u0 = u_hist[end] + (u_hist[end] - u_hist[end-1]) * 
             dgamma3 / (gamma3_hist[end] - gamma3_hist[end-1])
    elseif length(u_hist) == 1
        u0 = u_hist[end]
    else
        u0 = rand((Nx+1) * harmonics * 2)*.3
    end
    
    
    pde::Symbolics.Num = make_WaveEquation_1D(u_HB, g0, height, gamma, gamma3, forcing);
    

    expanded = expand_trig_jl(pde, t, omega)
    eqns = make_equations(expanded, harmonics, omega, t)
    sym_eqs = map(transform_sym(Nx,) ∘ Meta.parse ∘ string, eqns)
    
    resid = create_residual_function(sym_eqs, vars, Nx, 0, bc_conds);
    
    residual! = eval(resid)
    
    DiffMat, LaplCoeff = create_jac_blocks(eqns, var_exprs, harmonics)
    jacobian! = create_jacobian_function(Nx + 1, DiffMat, LaplCoeff, harmonics)
    
    # Create prototype by calling jacobian! once with dummy sparse matrix
    n = (Nx + 1) * harmonics * 2
    jac_prototype = spzeros(n, n)
    jacobian!(jac_prototype, u0, [stepx])
    
    func = NonlinearFunction(residual!; jac = jacobian!, jac_prototype = jac_prototype)
    prob = NonlinearProblem(func, u0, [stepx])
    
    solHB = solve(prob, FastShortcutNonlinearPolyalg(), reltol=1e-5, abstol=1e-5, maxiters=3000)
    u0 = solHB.u
    println(round(gamma3, sigdigits=6), " ", solHB.retcode)
    if solHB.retcode == ReturnCode.Success
        println("Succeeded!")
        flush(stdout)
        push!(u_hist, copy(solHB.u))
        push!(gamma3_hist, gamma3)
        dgamma3 = min(dgamma3 * 1.2, 0.01)
        
        gamma3 += dgamma3

        if gamma3 > gamma3_final
            gamma3 = gamma3_final
            break
        end
        
    else
        println("Failed! Going back to the previous gamma and reducing the step!")
        flush(stdout)
        dgamma3 = min(dgamma3 * 1.2, 0.005)
        gamma3 += dgamma3
        # dgamma3 /= 10
        # gamma3 += dgamma3
        dgamma3 < 1e-6 && break
    end


    
end

pde::Symbolics.Num = make_WaveEquation_1D(u_HB, g0, height, gamma, gamma3, forcing);
    
expanded = expand_trig_jl(pde, t, omega)
eqns = make_equations(expanded, harmonics, omega, t)
sym_eqs = map(transform_sym(Nx,) ∘ Meta.parse ∘ string, eqns)

resid = create_residual_function(sym_eqs, vars, Nx, 0, bc_conds);

residual! = eval(resid)

DiffMat, LaplCoeff = create_jac_blocks(eqns, var_exprs, harmonics)
jacobian! = create_jacobian_function(Nx + 1, DiffMat, LaplCoeff, harmonics)

# Create prototype by calling jacobian! once with dummy sparse matrix
n = (Nx + 1) * harmonics * 2
jac_prototype = spzeros(n, n)
jacobian!(jac_prototype, u0, [stepx])  # fills in the pattern

func = NonlinearFunction(residual!; jac = jacobian!, jac_prototype = jac_prototype)
prob = NonlinearProblem(func, u0, [stepx])

@time solHB = solve(prob, NewtonRaphson(linsolve = KLUFactorization()), reltol=1e-5, abstol=1e-5, maxiters=1000)

print(solHB.retcode)

coefficientsHB = [reshape(solHB.u[(k-1)*(Nx + 1)+1:k*(Nx + 1)], Nx+1) for k in 1:(2*harmonics)];

In [ ]:
eqns

## Visualization HB

In [ ]:
anim = @animate for t in 1:0.01:10
    if t % 100 == 0
        println(t)
    end
    u_new = zeros(length(grid))
        j = 1
        for i in 1:(2*harmonics)
            if isodd(i)
                u_new .+= coefficientsHB[i] .* sin(j * omega * t)
            else
                u_new .+= coefficientsHB[i] .* cos(j * omega * t)
                j += 1
            end
        end
    plot(u_new)
    title!("Wave Equation with H = $(harmonics), \$\\gamma\$ = $(gamma), \$\\gamma_3\$ = $(gamma3_final), \$\\omega\$ = $omega", titlefontsize=10)
    ylims!(-0.02, 0.02)
end

gif(anim, "HB_WE_1D.gif", fps=60)

# Fourier Transform Comparison

In [ ]:
t_steady0 = 19000
x_points = size(sol_u)[1]
spatial_grid = 1:x_points
coeffs::Vector{Vector{Float64}} = []
fs_time = fs = 1/0.01

#Finding amplitude at specific frequency
for h in 1:harmonics
    target = h * omega / (2 * pi)
    println("new target: ", target)
    push!(coeffs, spatial_grid |> length |> zeros)
    push!(coeffs, spatial_grid |> length |> zeros)
    for point_idx in 1:1:x_points
        u_at_point = sol_u[point_idx, t_steady0:end]
        F = fftshift(fft(u_at_point))
        freqs = fftshift(fftfreq(length(u_at_point), fs_time))
        (_, idx) = findmin(abs.(freqs .- target))
        #println(idx, ", ", F[idx], ", ", -2*imag(F[idx]), ", ", 2*real(F[idx]), ", ", target)
        coeffs[2 * h - 1][point_idx] = -2*real(F[idx])*(1/(size(sol_u)[2] - t_steady0)) # sin
        coeffs[2 * h][point_idx] = -2*imag(F[idx])*(1/(size(sol_u)[2] - t_steady0)) # cos
    end
end

In [ ]:
plots = Vector{Plots.Plot}(undef, 2 * harmonics)

for i in 1:(2*harmonics)
    title = ""
    if i % 2 == 1
        title = string("A", div(i + 1, 2))
    else
        title = string("B", div(i, 2))
    end
    p = plot(xleft:stepx:xright, coefficientsHB[i]; ylim=(-2.5, 2.5), title = title, label="HB Coeff") 
    plot!(p, xleft:stepx:xright, coeffs[i];  ylim=(-2, 2),label="FD coeff")
    plots[i] = p
end

plot(plots...; layout=(harmonics, 2), legend=false)

In [ ]:
function generateSpectrumMatrix()
    r1 = []
    r2 = []
    rows = 0
    for point_idx in 1:1:x_points
        u_at_point = sol_u[point_idx, t_steady0:end]
        F = fftshift(fft(u_at_point))
        freqs = fftshift(fftfreq(length(u_at_point), fs_time))
        rows = length(freqs)

        push!(r1, (-2 .* real.(F) .* (1/(size(sol_u)[2] - t_steady0)))...)
        push!(r2, (-2 .* imag.(F) .* (1/(size(sol_u)[2] - t_steady0)))...)
        

        # (_, idx) = findmin(abs.(freqs .- target))
        # #println(idx, ", ", F[idx], ", ", -2*imag(F[idx]), ", ", 2*real(F[idx]), ", ", target)
        # coeffs[2 * h - 1][point_idx] = -2*real(F[idx])*(1/(size(sol_u)[2] - t_steady0)) # sin
        # coeffs[2 * h][point_idx] = -2*imag(F[idx])*(1/(size(sol_u)[2] - t_steady0)) # cos
        
    end
    res1 = reshape(r1, (rows, div(length(r1), rows)))
    res2 = reshape(r2, (rows, div(length(r2), rows)))
    res1, res2
end
sins, coses = generateSpectrumMatrix()
plot(heatmap(sins), heatmap(coses); layout = (1, 2))